# LangChain 1.2+ Demo - 使用 HuggingFace 本地模型

这个 notebook 展示了 LangChain 1.2+ 版本与 HuggingFace 本地模型的使用方法。

**无需 API Key！** 所有模型都在本地运行。

## 功能演示

1. **基本的 LLM 调用** - 展示如何使用 LangChain 调用本地 HuggingFace 模型
2. **提示模板** - 演示如何使用提示模板格式化输入
3. **链式调用** - 展示 LLMChain 的使用方法
4. **多步骤处理** - 演示如何将多个步骤串联起来
5. **对话式交互** - 展示如何维护对话上下文

---

## 步骤 1: 安装依赖

首先需要安装 LangChain 1.2+ 版本和 HuggingFace 相关依赖。


In [1]:
# 安装 LangChain 1.2 和 HuggingFace 相关依赖
!pip install  langchain langchain-community langchain-core
!pip install transformers torch accelerate sentencepiece


## 步骤 2: 选择并加载 HuggingFace 模型

选择你想要使用的模型。这里提供了几个选项：

- **Qwen2.5-0.5B** (推荐，小模型，速度快) - `Qwen/Qwen2.5-0.5B-Instruct`
- **ChatGLM3-6B** (中文效果好) - `THUDM/chatglm3-6b`
- **Qwen2.5-1.5B** (平衡选择) - `Qwen/Qwen2.5-1.5B-Instruct`
- **其他模型** - 可以替换为任何 HuggingFace 上的模型

**注意：** 首次运行会下载模型，可能需要一些时间。模型会缓存在本地，后续运行会更快。


In [3]:
# 选择要使用的模型（可以修改为其他 HuggingFace 模型）
# MODEL_NAME = "Qwen/Qwen2.5-0.5B-Instruct"  # 小模型，适合快速测试
# MODEL_NAME = "THUDM/chatglm3-6b"  # 中文效果更好的模型
MODEL_NAME = "Qwen/Qwen2.5-1.5B-Instruct"  # 中等大小的模型

print(f"✅ 将使用模型: {MODEL_NAME}")
print("📥 首次运行会下载模型，请耐心等待...")


✅ 将使用模型: Qwen/Qwen2.5-1.5B-Instruct
📥 首次运行会下载模型，请耐心等待...


## 步骤 3: 导入必要的库并初始化模型


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
# from langchain.chains import LLMChain
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

print("✅ 所有库已导入！")
print(f"✅ 使用 LangChain 1.2+ 版本")
print(f"✅ PyTorch 版本: {torch.__version__}")

# 加载模型和分词器
print(f"\n📦 正在加载模型 {MODEL_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

# 根据设备选择加载方式
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🖥️  使用设备: {device}")

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    device_map="auto" if device == "cuda" else None,
    low_cpu_mem_usage=True
)

if device == "cpu":
    model = model.to(device)

# 创建 pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    do_sample=True,
    return_full_text=False,
    device=0 if device == "cuda" else -1
)

# 创建 LangChain LLM
llm = HuggingFacePipeline(pipeline=pipe)

print("✅ 模型加载完成！可以开始使用了。")

✅ 所有库已导入！
✅ 使用 LangChain 1.2+ 版本
✅ PyTorch 版本: 2.9.0+cu126

📦 正在加载模型 Qwen/Qwen2.5-1.5B-Instruct...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

🖥️  使用设备: cuda


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


---

## 演示 1: 基本的 LLM 调用

这是最简单的使用方式，直接调用 LLM 并获取回答。


In [ ]:
print("=" * 50)
print("演示 1: 基本的 LLM 调用")
print("=" * 50)

# 简单调用（注意：HuggingFacePipeline 返回字符串，不是对象）
response = llm.invoke("用一句话解释什么是人工智能？")
print(f"\n回答: {response}\n")


## 演示 2: 使用提示模板

提示模板可以帮助我们更好地组织和格式化输入，特别适合需要动态内容的场景。


In [ ]:
print("=" * 50)
print("演示 2: 使用提示模板")
print("=" * 50)

# 创建提示模板（使用字符串模板，因为 HuggingFacePipeline 接受字符串）
prompt = ChatPromptTemplate.from_template(
    "你是一个专业的技术顾问，擅长用简单易懂的方式解释技术概念。\n\n请解释一下 {technology} 是什么，并给出一个实际应用例子。"
)

# 格式化提示
formatted_prompt = prompt.format(technology="区块链")
response = llm.invoke(formatted_prompt)
print(f"\n回答: {response}\n")


## 演示 3: 链式调用 (LLMChain)

LLMChain 是 LangChain 的核心概念之一，它将提示模板和 LLM 组合成一个可重用的链。

In [ ]:
print("=" * 50)
print("演示 3: 链式调用 (LLMChain)")
print("=" * 50)

# 创建提示模板
prompt = ChatPromptTemplate.from_template(
    "将以下文本翻译成英文，并总结其主要内容：\n\n{text}"
)

# 创建链
chain = LLMChain(llm=llm, prompt=prompt)

# 执行链
result = chain.invoke({
    "text": "人工智能是计算机科学的一个分支，旨在创建能够执行通常需要人类智能的任务的系统。"
})

print(f"\n结果: {result['text']}\n")


## 演示 4: 多步骤处理

有时候我们需要将多个操作串联起来，形成一个处理流程。


In [ ]:
print("=" * 50)
print("演示 4: 多步骤处理")
print("=" * 50)

# 步骤1: 生成主题
step1_prompt = ChatPromptTemplate.from_template(
    "基于以下关键词生成一个技术主题：{keywords}"
)
step1_chain = LLMChain(llm=llm, prompt=step1_prompt)

# 步骤2: 基于主题生成内容
step2_prompt = ChatPromptTemplate.from_template(
    "为主题 '{topic}' 写一段简短的介绍（100字以内）"
)
step2_chain = LLMChain(llm=llm, prompt=step2_prompt)

# 执行多步骤
print("\n正在执行步骤1: 生成主题...")
topic_result = step1_chain.invoke({"keywords": "机器学习, 深度学习, 神经网络"})
topic = topic_result['text'].strip()

print("正在执行步骤2: 生成主题介绍...")
content_result = step2_chain.invoke({"topic": topic})

print(f"\n生成的主题: {topic}")
print(f"主题介绍: {content_result['text']}\n")


## 演示 5: 对话式交互

这个演示展示了如何维护对话上下文，实现多轮对话。


In [ ]:
print("=" * 50)
print("演示 5: 对话式交互")
print("=" * 50)

# 构建对话提示（HuggingFacePipeline 使用字符串输入）
conversation_history = "你是一个友好的AI助手，擅长回答技术问题。\n\n"

# 第一轮对话
print("\n第一轮对话:")
user_input1 = "什么是 LangChain？"
prompt1 = conversation_history + f"用户: {user_input1}\n助手:"
response1 = llm.invoke(prompt1)
print(f"用户: {user_input1}")
print(f"助手: {response1}\n")

# 第二轮对话（带上下文）
print("第二轮对话（带上下文）:")
conversation_history += f"用户: {user_input1}\n助手: {response1}\n\n"
user_input2 = "它有什么主要优势？"
prompt2 = conversation_history + f"用户: {user_input2}\n助手:"
response2 = llm.invoke(prompt2)
print(f"用户: {user_input2}")
print(f"助手: {response2}\n")


---

## 🎉 完成！

所有演示已完成。你可以：

1. **修改参数** - 尝试不同的 temperature、model 等参数
2. **自定义提示** - 修改提示模板来适应你的需求
3. **扩展功能** - 添加更多步骤或集成其他 LangChain 组件

### 下一步探索

- 向量数据库集成（如 Chroma、Pinecone）
- 工具和代理（Agents）
- 文档加载和处理
- 记忆（Memory）功能
- 流式输出（Streaming）

### 参考资源

- [LangChain 官方文档](https://python.langchain.com/)
- [LangChain GitHub](https://github.com/langchain-ai/langchain)
